In [5]:
import numpy as np 
import pandas as pd 
import os
import sys
import platform
system = platform.system()
if system == 'Linux':
    sys.path.append('/home/lucas/Documents/Log_Analysis')
    sys.path.append('/home/lucas/Documents/Log_Analysis/Battery')
elif system == 'Darwin':
    sys.path.append('/Users/Lucas/Documents/Travail/Yuneec/LogAnalysis')
    sys.path.append('/Users/Lucas/Documents/Travail/Yuneec/LogAnalysis/Battery')
import analog
import battery
from battery import OCVcurve, segment
from analog import logextract
import datetime
import matplotlib.pyplot as plt
%matplotlib notebook
import csv

In [11]:
if system == 'Linux':
    log_path = '/home/lucas/Documents/Log_Analysis/Battery/Battery 9 (RIP)'
else : 
    log_path = '/Users/Lucas/Documents/Travail/Yuneec/LogAnalysis/Battery/Battery 9 (RIP)'
print(os.listdir(log_path))
if system == 'Linux':
    file1 = f'{log_path}/Discharge 200mA/2019-09-20_1_LiHV - Discharge.csv'
    #file2 = f'{log_path}/2019-09-06_1_LiHV - Discharge-part2.csv'
    #file3 = f'{log_path}/2019-09-09_1_LiHV - Discharge-part3.csv'
    file1_charge = f'{log_path}/Charge 200mA/2019-09-26_1_LiHV - Normal-Charge_200mA.csv' 
    #file2_charge = f'{log_path}/Charge 200mA/2019-09-10_1_LiHV - Charge-part2.csv' 
    #file3_charge = f'{log_path}/Charge 200mA/2019-09-11_1_LiHV - Charge-part3.csv' 
else : 
    file1 = f'{log_path}/Discharge 200mA/SOCvsOCV_discharge200mA.csv'

['.DS_Store', 'Discharge 200mA']


In [12]:
seg1 = segment(path2csv=file1,mode='discharge')
#seg2 = segment(path2csv=file2,mode='discharge')
#seg3 = segment(path2csv=file3,mode='discharge')

KeyError: 'Time [hh:mm:ss.SSS]'

In [ ]:
seg1_charge = segment(path2csv=file1_charge,mode='charge')
#seg2_charge = segment(path2csv=file2_charge,mode='charge')
#seg3_charge = segment(path2csv=file3_charge,mode='charge')

In [ ]:
print(seg1_charge.cell_voltage)
#print(seg2_charge.cell_voltage)
#print(seg3_charge.cell_voltage)

In [ ]:
plt.figure()
plt.subplot(231)
seg1.vplot()
plt.ylabel('cell voltage (V)')

#plt.subplot(232)
#seg2.vplot()

#plt.subplot(233)
#seg3.vplot()

plt.subplot(234)
seg1_charge.vplot()
plt.xlabel('time (s)')
plt.ylabel('cell voltage (V)')

#plt.subplot(235)
#seg2_charge.vplot()
#plt.xlabel('time (s)')
#plt.ylabel('cell voltage (V)')

#plt.subplot(236)
#seg3_charge.vplot()
#plt.xlabel('time (s)')
plt.show()

In [ ]:
#discharge = seg1.concatenate(seg2.concatenate(seg3))
discharge = seg1

In [ ]:
#charge = seg1_charge.concatenate(seg2_charge.concatenate(seg3_charge))
charge = seg1_charge

In [ ]:
plt.figure()
plt.subplot(221)
discharge.vplot(label='discharge')
plt.ylabel('voltage (V)')
plt.title('Discharge')
plt.subplot(222)
charge.vplot(label='charge')
plt.title('Charge')
plt.subplot(223)
discharge.cplot(label='discharge')
plt.xlabel('time (s)')
plt.ylabel('current (A)')

plt.subplot(224)
charge.cplot(label='charge')
plt.xlabel('time (s)')
plt.show()

print(discharge.mode)

In [ ]:
charge.count_charge(0)
discharge.count_charge(1)

print(f'Total discharge capacity is {round(discharge.Q/3.6,2)} mAh')
print(f'Total charge capacity is {round(charge.Q/3.6,2)} mAh')
eta = discharge.Q/charge.Q
print(eta)

In [ ]:
%matplotlib notebook
plt.figure()
plt.subplot(121)
discharge.zplot()
plt.xlabel('time (s)')
plt.ylabel('state of charge (-)')
plt.title('Cell discharging curve')
plt.subplot(122)
charge.zplot()
plt.xlabel('time (s)')
plt.title('Cell charging curve')
plt.show()

In [ ]:
charge_OCV = OCVcurve()
charge_OCV.fromsegment(charge)
discharge_OCV = OCVcurve()
discharge_OCV.fromsegment(discharge)

R0 = 0.0123
charge_OCV.intrescorr(R0)
discharge_OCV.intrescorr(R0)

In [ ]:
plt.figure()
#charge_OCV.plot()
discharge_OCV.plot(color='orange')
#plt.legend(['charge','discharge'])
#plt.grid()

In [ ]:
csv_file = open(f'Battery 9 (RIP)/Charge 200mA/SOCvsOCV_{charge_OCV.mode}200mA.csv','w')
writer = csv.writer(csv_file)
writer.writerow(['SOC','OCV'])
for k in range(len(charge_OCV.SOC)):
    writer.writerow([charge_OCV.SOC.values[k],charge_OCV.OCV.values[k]])

In [ ]:
csv_file = open(f'Battery 9 (RIP)/Discharge 200mA/SOCvsOCV_{discharge_OCV.mode}200mA.csv','w')
writer = csv.writer(csv_file)
writer.writerow(['SOC','OCV'])
for k in range(len(discharge_OCV.SOC)):
    writer.writerow([discharge_OCV.SOC.values[k],discharge_OCV.OCV.values[k]])

In [ ]:
p = np.polyfit(discharge_OCV.SOC,discharge_OCV.OCV,11)
print(p)

In [ ]:
plt.figure()
discharge_OCV.plot(color='orange')
plt.plot(discharge_OCV.SOC,np.polyval(p,discharge_OCV.SOC))
plt.show()

In [ ]:
print(np.polyval(p,0.01))
print(np.polyval(p,1.0))
print(3.339*4)

### How much off are we ?

In [ ]:
# maximum SOC change = 0.05 per iteration
# From 1.0, the SOC changes to 0.95. We compute OCV(0.95) with the C matrix computed at 1.0. How much off are we ? 
bat = battery.Thevenin(1.0)

## Interpolation method for initializing SOC

In [ ]:
zseg_list = [0,0.04,0.6,1]
discharge_OCV.OCVfromSOC(zseg_list)

In [ ]:
plt.figure()
plt.plot(discharge_OCV.SOC, discharge_OCV.OCV,color='orange')
for zseg in zseg_list:
    plt.plot(zseg,discharge_OCV.OCVfromSOC(zseg),color='red',Marker='x')
plt.plot(zseg_list, discharge_OCV.OCVfromSOC(zseg_list))
plt.grid()
plt.show()

In [ ]:
plt.figure()
plt.plot(discharge_OCV.OCV,discharge_OCV.SOC,color='orange')
for zseg in zseg_list:
    plt.plot(discharge_OCV.OCVfromSOC(zseg),zseg,color='red',Marker='x')
plt.plot(discharge_OCV.OCVfromSOC(zseg_list),zseg_list)
plt.grid()
plt.show()

In [ ]:
OCV = 17.21/4
OCV_list = discharge_OCV.OCVfromSOC(zseg_list)
SOC_list = zseg_list

if (OCV >= OCV_list[0] and OCV<=OCV_list[3]):
    if (OCV >= OCV_list[0] and OCV<=OCV_list[1]):
        SOC = (SOC_list[1]-SOC_list[0])/(OCV_list[1] - OCV_list[0])*(OCV-OCV_list[0])
        print('part 1')
    elif (OCV > OCV_list[1] and OCV<=OCV_list[2]):
        SOC =  (SOC_list[2]-SOC_list[1])/(OCV_list[2] - OCV_list[1])*(OCV-OCV_list[1]) + SOC_list[1]
        print('part 2')
    elif (OCV > OCV_list[2] and OCV<=OCV_list[3]):
        SOC = (SOC_list[3]-SOC_list[2])/(OCV_list[3] - OCV_list[2])*(OCV-OCV_list[2]) + SOC_list[2]
        print('part 3')
print(SOC)
print(discharge_OCV.SOCfromOCV(OCV))

In [ ]:
discharge_OCV.SOCfromOCV(14.67/4)

In [ ]:
discharge_OCV.OCVfromSOC(0.1)*4

In [ ]:
discharge_OCV.OCVfromSOC(0.0)*4

In [ ]:
(14.67-13.18)/0.01

In [ ]:
(14.89-12.35)/0.01

In [ ]:
18.746/3